# Homework 3
## 211622600 - George Jobran
## 

In [131]:
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.probability import FreqDist
from nltk.corpus import words
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from numpy import dot
import pandas as pd
from numpy.linalg import norm
import operator

In [6]:
def match_time(string):
    return [res for res in word_tokenize(string) if re.findall('^(0[0-9]|1[0-9]|2[0-3]):([0-5][0-9])$',res)]
match_time("asd:asd 123:123 12:10 4: :00 00:00 ")

['12:10', '00:00']

In [7]:
def match_phonenumber(string):
    return [res for res in word_tokenize(string) if re.findall('^[+]?[(]?0[0-9]{1,2}[)]?[-\s.]?([0-9]{3})[-\s.]?[0-9]{3,6}$',res)]
match_phonenumber('0525080784 (052) 5080784 052-508-0784 052-5080784 +972 525080784 048538802 (04) 8538802 (04)8538802')

['0525080784', '052-508-0784', '052-5080784', '048538802']

In [8]:
def match_comment(string):
    return re.findall('^/\*[\S\s]*?\*/$',string)
match_comment("/* test */")

['/* test */']

In [9]:
def match_englishnum(string):
    return [res for res in word_tokenize(string) if re.findall('^Thirty(-one|-two|-three|-four|-five|-six|-seven|-eight|-nine|-ten)?$',res)]
match_englishnum("Thirty Thirty-two asd")

['Thirty', 'Thirty-two']

In [10]:
def match_date(string):
    return [res for res in word_tokenize(string) if re.findall('^(\d{4})-(0[13578]|1[02])-(0[1-9]|[12]\d|3[01])$|^(\d{4})-(0[13456789]|1[012])-(0[1-9]|[12]\d|30)$|^(\d{4})-(02-(0[1-9]|1\d|2[0-8]))$|^(([02468][048]|[13579][26])00)$|^(\d{2})([13579][26]|0[48]|[2468][048])-(02-(29))$',res)]
match_date('2016-04-20 2016-04-31 2020-02-29 2021-02-29')

['2016-04-20', '2020-02-29']

## Question 2

### A

In [ ]:
data = open("misspellings_and_corrections.txt", "r").read()

In [34]:
def calculate_diff(string):
    matches = re.findall('<ERR targ=([^>]*)>([^<]*)</ERR>', string)
    result = []
    for match in matches:
        targ = match[0]
        error = match[1].strip()
        result.append(str(error) + "=>" + str(targ) + " distance: " + str(nltk.edit_distance(targ,error)))
    return result
#     return ["" + targ + "=>" + error + " distance: " + str(nltk.edit_distance(match[0],match[1])) for match in matches] or None
calculated_data = calculate_diff(data)
print(calculated_data)

['siter=>sister distance: 1', 'siter=>sister distance: 1', 'go=>goes distance: 2', 'some times=>sometimes distance: 1', 'some times=>sometimes distance: 1', 'clob=>club distance: 1', 'bell ringing=>bellringing distance: 1', 'wakh=>watch distance: 2', 'frount=>front distance: 1', 'sexeon=>second distance: 3', 'wach=>watch distance: 1', 'wach=>watch distance: 1', 'cow Boys=>cowboys distance: 2', 'some times=>sometimes distance: 1', 'colbe=>club distance: 3', 'wach=>watch distance: 1', 'wach=>watch distance: 1', 'thing=>think distance: 1', 'tv=>TV distance: 2', 'squar=>square distance: 1', 'iyes=>eyes distance: 1', "oclock=>o'clock distance: 1", 'nock=>knocked distance: 3', 'a=>at distance: 1', "oclock=>o'clock distance: 1", 'kild=>killed distance: 2', 'see=>saw distance: 2', 'bean=>been distance: 1', 'nock=>knocked distance: 3', 'cald=>called distance: 2', 'cam=>came distance: 1', 'killd=>killed distance: 1', 'the=>there distance: 2', 'Here=>Her distance: 1', 'iyes=>eyes distance: 1', 'h

### B

In [97]:
over_2_list = [word.split("=>")[1].split(" distance: ")[0] for word in calculated_data if int(word.split(":")[1].strip()) > 2]
equal_2_list = [word.split("=>")[1].split(" distance: ")[0]  for word in calculated_data if int(word.split(":")[1].strip()) == 2]
equal_1_list = [word.split("=>")[1].split(" distance: ")[0]  for word in calculated_data if int(word.split(":")[1].strip()) == 1]
over_2 = len(over_2_list)
less_than_2 = len(equal_2_list) + len(equal_1_list)

print("Over 2 count: " + str(over_2))
print("Less than 2 count: " + str(less_than_2))
print("Over 2 Percentage: " + str(100 * over_2 / (over_2 + less_than_2)))
print("Less than 2 Percentage: " + str(100 * less_than_2 / (over_2 + less_than_2)))

Over 2 count: 383
Less than 2 count: 2217
Over 2 Percentage: 14.73076923076923
Less than 2 Percentage: 85.26923076923077


In [93]:
over_2_freqdist = FreqDist(over_2_list)
over_2_freqdist.most_common(10)

[('?', 20),
 ('farmer', 13),
 ('castle', 9),
 ('stopped', 6),
 ('knocked', 5),
 ('teddy boy', 5),
 ('Teddy boy', 4),
 ('walked', 4),
 ('alright', 4),
 ('they', 4)]

In [94]:
equal_2_freqdist = FreqDist(equal_2_list)
equal_2_freqdist.most_common(10)

[('their', 24),
 ('there', 20),
 ('said', 12),
 ('vet', 11),
 ('here', 10),
 ('know', 9),
 ('want', 9),
 ('looked', 8),
 ('killed', 7),
 ('There', 6)]

In [95]:
equal_1_freqdist = FreqDist(equal_1_list)
equal_1_freqdist.most_common(10)

[('too', 23),
 ('to', 21),
 ('James', 21),
 ("don't", 20),
 ('her', 18),
 ('what', 16),
 ('got', 15),
 ('went', 13),
 ('off', 13),
 ('his', 12)]

In [66]:
def find_equal_1(error_word):
    results=[]
    for word in set(words.words()):
        if nltk.edit_distance(error_word.lower(),word.lower()) == 1:
            results.append(word)
    return results
find_equal_1('hello')

['chello', 'cello', 'helio', 'helluo', 'hollo', 'helly', 'hell']

In [99]:
wrong_words = [word.split("=>")[0] for word in calculated_data]

wrong_words_50=wrong_words[:50]
count=0
for word in set(wrong_words_50):
    count += len(find_equal_1(word))
average = count / 50
print("The average result count per word is:" + str(average))

The average result count per word is:13.72


In [100]:
count = 0
for word in wrong_words_50:
    for correct_word in set(words.words()):
        if nltk.edit_distance(correct_word,word.lower()) == 0:
            count += 1
print("The amount of correct words is: " + str(count))

The amount of correct words is: 18


## Question 3

In [103]:
# train documents:
news_group = fetch_20newsgroups(subset='train')

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

In [104]:
news_group_test = fetch_20newsgroups(subset='test')
news_group_test.filenames

array(['C:\\Users\\win10\\scikit_learn_data\\20news_home\\20news-bydate-test\\rec.autos\\103343',
       'C:\\Users\\win10\\scikit_learn_data\\20news_home\\20news-bydate-test\\comp.windows.x\\67445',
       'C:\\Users\\win10\\scikit_learn_data\\20news_home\\20news-bydate-test\\alt.atheism\\53603',
       ...,
       'C:\\Users\\win10\\scikit_learn_data\\20news_home\\20news-bydate-test\\rec.sport.baseball\\104986',
       'C:\\Users\\win10\\scikit_learn_data\\20news_home\\20news-bydate-test\\misc.forsale\\76564',
       'C:\\Users\\win10\\scikit_learn_data\\20news_home\\20news-bydate-test\\soc.religion.christian\\21696'],
      dtype='<U94')

In [112]:
text_data = np.array(news_group_data)
test_data = np.array(news_group_test.data)

count_vectorizer = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
count_vectorizer.fit(text_data)

count_vector = count_vectorizer.transform(text_data)
count_vector_test = count_vectorizer.transform(test_data)

tfidf_vectorizer = TfidfVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
tfidf_vectorizer.fit(text_data)

tfidf_vector = tfidf_vectorizer.transform(text_data)
tfidf_vector_test = tfidf_vectorizer.transform(test_data)

In [126]:
def cosine(a,b):
    angle = dot(a, b)/(norm(a)*norm(b))
    return angle

class NearestNeighbors:
    def __init__(self, num_neighbors):
        self.num_neighbors = num_neighbors

    def fit(self, training_data):
        self.training_data = training_data

    def find_neighbors(self, new_sample):
        similarities, num_training_samples = {}, len(self.training_data)
        for i in range(num_training_samples-1):
            similarity = cosine(a=self.training_data[i],b=new_sample)
            similarities[i] = similarity
        sorted_similarities = sorted(similarities.items(), key=operator.itemgetter(1),reverse=True)
        sorted_similarities_dict= dict(sorted_similarities)
        return list(sorted_similarities_dict.keys())[:self.num_neighbors]

In [132]:
start_time = time.time()
results_dict={'index':[], 'actual_label':[],'predicted_label':[]}
results_df=pd.DataFrame(results_dict)
k=1
knn_model=NearestNeighbors(k)
knn_model.fit(tfidf_vector.toarray())

for i in range(len(tfidf_vector_test.toarray())):
    nearest_neighbors_indices= knn_model.find_neighbors(tfidf_vector_test.toarray()[i])
    nearest_neighbors_labels=list(news_group_target[nearest_neighbors_indices])
    majority_label = max(set(nearest_neighbors_labels), key = nearest_neighbors_labels.count)
    new_row = {'index':i, 'actual_label':news_group_test.target_names[news_group_test.target[i]], 'predicted_label':news_group_target_names[majority_label]}
    results_df = results_df.append(new_row, ignore_index=True)
end_time = time.time()
print(f"Runtime of the program is {end_time - start_time}")
results_df.to_csv(r'knn_results.csv',index=False)
results_df


C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df 

C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df 

C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df 

C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df 

C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df 

...

C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df 

C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)
C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df 

Runtime of the program is 3102.8144991397858


C:\Users\win10\AppData\Local\Temp\ipykernel_24904\1946245275.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(new_row, ignore_index=True)


,index,actual_label,predicted_label
0,0.0,rec.autos,sci.electronics
1,1.0,comp.windows.x,rec.sport.hockey
2,2.0,alt.atheism,alt.atheism
3,3.0,talk.politics.mideast,alt.atheism
4,4.0,talk.religion.misc,alt.atheism
...,...,...,...
7527,7527.0,sci.space,sci.space
7528,7528.0,comp.sys.mac.hardware,comp.windows.x
7529,7529.0,rec.sport.baseball,rec.sport.baseball
7530,7530.0,misc.forsale,alt.atheism
